In [1]:
%run basics_sensorSignal.py
%matplotlib

Using matplotlib backend: MacOSX


#### TAN = c(NH$_3$) + c(NH$_4^+$)

sensor response is sigmoidal. It mainly depends on the response time 

In [4]:
c_nh3, step = 2.5, 0.25
t = np.linspace(0, 60, num=int(100/step+1))

par_sens1 = dict({'max conc': c_nh3, 'response time': 10., 'background signal': 5E-6})
par_sens2 = dict({'max conc': c_nh3, 'response time': 50., 'background signal': 5E-6})
# ...........................................
dfSig1 = _sensor_response(t=t, par_sens=par_sens1, y=0, function='gompertz',
                          direction='increase')
dfSig2 = _sensor_response(t=t, par_sens=par_sens2, y=0, function='gompertz',
                          direction='increase')
dfSig3 = _sensor_response(t=t, par_sens=par_sens1, y=c_nh3, function='gompertz',
                          direction='decline')
dfSig4 = _sensor_response(t=t, par_sens=par_sens2, y=c_nh3, function='gompertz',
                          direction='decline')


# .........................................................................
# plot sensor response
fig, ax = plt.subplots(figsize=(7,4))
ax.set_title('Sensor response', loc='left')

# sensor 1
ax.plot(t, dfSig1, 'k', label='fast response')
ax.axhline(dfSig1['signal'].max()*0.1, lw=0.5, ls=':', color='darkorange'), 
ax.axhline(dfSig1['signal'].max()*0.9, lw=0.5, ls=':', color='darkorange')

# sensor 2
ax.plot(t, dfSig2, ':k', label='slow response')
ax.axhline(dfSig2['signal'].max()*0.1, lw=0.5, ls=':', color='crimson'), 
ax.axhline(dfSig2['signal'].max()*0.9, lw=0.5, ls=':', color='crimson')

# sensor 3
ax.plot(t, dfSig3, 'slategrey', label='fast response - decline')

# sensor 4
ax.plot(t, dfSig4, color='slategrey', ls=':', label='slow response - decline')

ax.set_xlabel('Time / s'), ax.set_ylabel('Signal / V')
plt.legend(loc='lower right', bbox_to_anchor=(1., 0.35))
plt.xlim(-5, t[-1]*1.05)
sns.despine()

sensor signal depends on concentration as well

In [8]:
start_time, end_time = 0, 60
nsample, nper = 10*(end_time-start_time), 3
dfconc1, D1, P1, t1 = _target_concentration(c_nh3=c_nh3, tstart=start_time, tstop=end_time)
dfconc2, D2, P2, t2 = _target_concentration(N=nsample, nP=nper, c_nh3=c_nh3, 
                                            tstart=start_time, tstop=end_time)

# .................................
fs = 10
fig1, ax1 = plt.subplots(figsize=(7,3))
ax1.set_xlabel('Time / s', fontsize=fs)
ax1.set_ylabel('Analyte concentration', fontsize=fs)
ax1.plot(dfconc1, color='k', lw=1., marker='.', fillstyle='none', 
         label='N:{:.0f}, period: {:.0f}s, freq: 1'.format(end_time-start_time, P1))
ax1.plot(dfconc2, color='darkorange', marker='.', fillstyle='none', lw=0.75,
         label='N:{:.0f}, period: {:.0f}s, freq: {:.0f}'.format(nsample, P1, nper))
ax1.legend(loc=0, fontsize=fs*0.7) #, ax1.set_ylim(-0.05, 1.6)
plt.tight_layout(), sns.despine()
plt.show()

time to combine

signal ~ concentration(t) * response time * some other factors

In [1]:
# gloabl parameter
c_nh3 = 2.5                          # maximal ammonia concentration 
# concentration fluctuation
tstart, tend, step = 0, 60*3, 0.05   # sec; time range of fluctuation
nP = 3                               # number of cycles per period
toc = (tend-tstart) / (2*nP)         # time of change; when concentration changes

# sensor settings
tstart_sens = 0                      # sec; start time range of sensor response            
t_resp = 20.                         # sensor response time
t_sample = 1/2*toc                   # sampling rate of the sensor; sec

# ........................................................................................
# target concentration
dfconc, D, P, trange = _target_concentration(nP=nP, c_nh3=c_nh3, tstart=tstart, tstop=tend)
print('puls width: ', D)

# .............................................
# sensor time range
sens_time0 = np.linspace(0, D, num=int((D - tstart_sens) / step + 1))

# .............................................
# plotting 
par_meas = dict({'start':tstart, 'stop':tend})
par_sens = dict({'max conc': c_nh3, 'response time': t_resp, 'background signal': 5E-5})
arg_fig = dict({'figsize': (6,3), 'fontsize': 10,
                'title': 'sensor response to fluctuating analyte concentration'})

NameError: name '_target_concentration' is not defined

In [532]:
fig, ax, dfdata = plot_sensorresponse(dfconc=dfconc, sens_time0=sens_time0, step=step,
                                      D=D, tstart_sens=tstart_sens, par_meas=par_meas, 
                                      par_sens=par_sens, arg_fig=arg_fig, plotCheck=False)

#sampling rate / scanning rate of the sensor
z, ls_samp, n = t_sample, list(), 0
while z < tend:
    z = t_sample*n
    n +=1
    ls_samp.append(z)
    if z < tend:
        pass #ax.axvline(z, color='darkorange', lw=0.5)

### how to extract data?
1) geometrical approach - intersection of 2 curves (easy when both curves are actual curves)

2) analytical approach

In [298]:
### geometrical approach
from shapely.geometry import LineString, Point

In [493]:
x = np.arange(0, 1000)
f = np.arange(0, 1000)
g = np.sin(np.arange(0, 10, 0.01) * 2) * 1000

plt.plot(x, f)
plt.plot(x, g)

# convert curves into LineString
first_line = LineString(np.column_stack((x, f)))
second_line = LineString(np.column_stack((x, g)))
intersection = first_line.intersection(second_line)

if intersection.geom_type == 'MultiPoint':
    plt.plot(*LineString(intersection).xy, 'o')
elif intersection.geom_type == 'Point':
    plt.plot(*intersection.xy, 'o')
    
# works when we do have 2 functions and not a straight line

In [533]:
### analytical approach
dfdata.index = [round(l, 2) for l in dfdata.index ]

#closest value
ls_samp_fit = list(map(lambda s: min(dfdata.index, key=lambda x:abs(x-s)), ls_samp))


#add identified intersetion points
ax.scatter(dfdata.loc[ls_samp_fit].index, dfdata.loc[ls_samp_fit]['signal'].to_numpy(),
           marker='s', s=12, color='#C51D74')

In [531]:
fig1 = plt.figure(figsize=arg_fig['figsize'])
fig1.canvas.set_window_title('c(t) depending on sensor sampling rate')
ax1 = fig1.add_subplot()

ax1.plot(dfconc, lw=1., ls=':', color='gray', label='target concentration')
ax1.scatter(dfdata.loc[ls_samp_fit].index, dfdata.loc[ls_samp_fit]['signal'].to_numpy(),
           marker='s', s=12, color='#C51D74', label='recorded concentration')
ax1.plot(dfdata.loc[ls_samp_fit], ls='-.', lw=1., color='#C51D74')
ax1.legend(loc=0, fontsize=8, frameon=True, fancybox=True)

ax1.set_ylabel('Analyte concentration $c(t)$')
ax1.set_xlabel('Time /s')
sns.despine()
plt.tight_layout()